In [1]:
# --- Set location of XASNet code
import sys
#sys.path.append('/home/samjhall/github/XASNet-XAI/src')
sys.path.append('D:\github\XASNet-XAI\src')
# --- Standard libraries
import os.path as osp
import pandas as pd
import numpy as np
import pickle as pkl
# --- Matplotlib
import matplotlib.pyplot as plt
import imageio
# --- PyTorch and PyG
import torch
from torch_geometric.loader import DataLoader
# --- XASNet
from XASNet.data import QM9_XAS
from XASNet.data import save_split
from XASNet.models import XASNet_GNN, XASNet_GAT, XASNet_GraphNet
from XASNet.trainer import GNNTrainer

In [2]:
# --- Load in the dataset
root = './XASNet-data/mol_dataset.pt'
go_spec = QM9_XAS(root=root,
                  raw_dir='./XASNet-data/',
                  spectra=[])

In [3]:
# --- Print details of the dataset
print(go_spec)
print('------------')
print(f'Number of graphs: {len(go_spec)}')
print(f'Number of features: {go_spec.num_features}')
print('')

# --- Print details of the first molecule/graph in dataset
data = go_spec[0]

print(data)
print('------------')
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

QM9_XAS(319)
------------
Number of graphs: 319
Number of features: 15

Data(x=[30, 15], edge_index=[2, 76], edge_attr=[76, 6], spectrum=[200], idx=[1], smiles='[c:0]12[c:4]3[c:8]4[c:10]5[cH:11][cH:14][c:15]6[c:13]4[c:17]4[c:19]([cH:18][cH:16]6)[cH:20][c:22]([OH:25])[c:23]([c:21]14)[CH2:24][CH:1]1[C:2]2([CH:3]=[CH:5][C:6]32[CH:7]([CH:9]5[C:12](=[O:26])[OH:27])[O:29]2)[O:28]1')
------------
Number of nodes: 30
Number of edges: 76
Average node degree: 2.53
Has isolated nodes: False
Has self loops: False
Is undirected: True


In [4]:
# --- Create spilt file with the dataset
# --- split into test, validation and test datasets
idxs = save_split(
    path='./raw/xasnet-mol-split.npz',
    ndata=len(go_spec),
    ntrain=252,
    nval=28,
    ntest=39,
    save_split=True,
    shuffle=True, 
    print_nsample=True
)

In [5]:
# --- Create variables for each dataset split
train_go = [go_spec[i] for i in idxs['train']]
val_go = [go_spec[i] for i in idxs['val']]
test_go = [go_spec[i] for i in idxs['test']]

# --- Save datasets splits into dataloaders
train_loader = DataLoader(train_go, batch_size=252, shuffle=True)
val_loader = DataLoader(val_go, batch_size=28, shuffle=True)
test_loader = DataLoader(test_go, batch_size=39, shuffle=False)

print(f'Training dataset length: {len(train_go)}, compiled in {len(train_loader)} loaders')
print(f'Validation dataset length: {len(val_go)}, compiled in {len(val_loader)} loaders')
print(f'Test dataset length: {len(test_go)}, compiled in {len(test_loader)} loaders')

Training dataset length: 252, compiled in 1 loaders
Validation dataset length: 28, compiled in 1 loaders
Test dataset length: 39, compiled in 1 loaders


In [6]:
# --- Save the dataloader to a file
torch.save(test_go, './XASNet-data/test_mol_dataset.pt')

In [7]:
# --- Define cost functions
def RSE_loss(prediction, target):
    dE = (300 - 280) / 200
    nom = torch.sum(dE*torch.pow((target-prediction), 2))
    denom = torch.sum(dE*target)
    return torch.sqrt(nom) / denom 

def RMSE(prediction, target):
    return torch.sqrt(torch.mean((target - prediction)**2))

In [8]:
# --- Set name for ML model
model_name = 'xasnet_model'
# --- Set number of epochs to run
num_epochs = 500
# --- Set the learning rate 
lr = 0.01
# --- Milestones to reduce learning rate in steps 
milestones = np.arange(10, 100, 10).tolist()

##### XASNet_GNN

In [9]:
# --- Set device for model to run on
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# --- Create the type of ML model you want to run
xasnet_gnn = XASNet_GNN(
    gnn_name = 'gcn', # model type
    in_channels = [15, 256, 128], # input nodes for each layer
    out_channels = [256, 128, 64], # output nodes for each layer
    num_targets = 200, # nodes for final output
    num_layers = 3, # number of total layers
    heads = 1
).to(device)

# --- Location to save model
path_to_model = osp.join('./best_model,', model_name)

# --- Check if there is an already existing model
if osp.exists(path_to_model):
    xasnet_gnn.load_state_dict(torch.load(path_to_model))
else:
    print('Model is not loaded.')

Model is not loaded.


In [10]:
# --- View the details of the created model
print(xasnet_gnn)
print('----')
print(f' Model will be trained on: {device}')

XASNet_GNN(
  (batch_norms): ModuleList()
  (interaction_layers): ModuleList(
    (0): GCNConv(15, 256)
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): GCNConv(256, 128)
    (4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): GCNConv(128, 64)
  )
  (dropout): Dropout(p=0.6, inplace=False)
  (out): Linear(in_features=64, out_features=200, bias=True)
)
----
 Model will be trained on: cpu


#### XASNet_GAT

In [1]:
# --- Set device for model to run on
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# --- Create the type of ML model you want to run
xasnet_gat = XASNet_GAT(
    gat_type = 'gatv2_custom', # model type
    node_features_dim = 15,
    in_channels = [512, 512, 256, 128], # input nodes for each layer
    out_channels = [512, 256, 128, 200], # output nodes for each layer
    targets = 200, # nodes for final output
    n_layers = 3, # number of total layers
    n_heads = 1,
    use_residuals = True,
    use_jk = True
).to(device)

# --- Location to save model
path_to_model = osp.join('./best_model,', model_name)

# --- Check if there is an already existing model
if osp.exists(path_to_model):
    xasnet_gnn.load_state_dict(torch.load(path_to_model))
else:
    print('Model is not loaded.')

NameError: name 'torch' is not defined

In [2]:
xasnet_gat

NameError: name 'xasnet_gat' is not defined

#### XASNet_GraphNet

In [27]:
# --- Set device for model to run on
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# --- Create the type of ML model you want to run
xasnet_graphnet = XASNet_GraphNet(
    node_dim = 15,
    edge_dim = 6,
    hidden_channels = 512,
    out_channels = 200, # output nodes for each layer
    gat_hidd = 512,
    gat_out = 100,
    n_targets = 200, # nodes for final output
    n_layers = 3 # number of total layers
).to(device)

# --- Location to save model
path_to_model = osp.join('./best_model,', model_name)

# --- Check if there is an already existing model
if osp.exists(path_to_model):
    xasnet_gnn.load_state_dict(torch.load(path_to_model))
else:
    print('Model is not loaded.')

Model is not loaded.


In [28]:
xasnet_graphnet

XASNet_GraphNet(
  (graphnets): ModuleList(
    (0): GraphNetwork(
      (gatencoder): GATEncoder(
        (gats): ModuleList(
          (0): GATv2Conv(15, 512, heads=3)
          (1): ReLU(inplace=True)
          (2): GATv2Conv(1536, 512, heads=3)
          (3): ReLU(inplace=True)
          (4): GATv2Conv(1536, 512, heads=3)
          (5): ReLU(inplace=True)
          (6): GATv2Conv(1536, 100, heads=1)
        )
      )
      (node_model): NodeModel(
        (mlp): Sequential(
          (0): Linear(in_features=127, out_features=512, bias=True)
          (1): ReLU(inplace=True)
          (2): Linear(in_features=512, out_features=200, bias=True)
          (3): ReLU(inplace=True)
          (4): LayerNorm((200,), eps=1e-05, elementwise_affine=True)
        )
      )
      (edge_model): EdgeModel(
        (mlp): Sequential(
          (0): Linear(in_features=506, out_features=512, bias=True)
          (1): ReLU(inplace=True)
          (2): Linear(in_features=512, out_features=200, bias=True

#### Train Model

In [11]:
chosen_model = xasnet_gnn

In [12]:
# --- Set additional ML parameters
optimizer = torch.optim.AdamW(chosen_model.parameters(), lr=lr)
loss_fn = torch.nn.L1Loss()
loss_fn2 = torch.nn.MSELoss()
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, 
                                                 milestones=milestones,
                                                 gamma=0.8)

In [13]:
# --- Create trainier
trainer = GNNTrainer(model = chosen_model,
                     model_name = model_name,
                     device = device,
                     metric_path = './metrics')

In [15]:
# --- Train the ML model
trainer.train_val(train_loader, val_loader, optimizer, RMSE,
                  scheduler, num_epochs, write_every=25, train_graphnet=False)

  0%|          | 0/500 [00:00<?, ?it/s]c:\Users\a3782\AppData\Local\anaconda3\envs\xasnet-xai\lib\site-packages\torch\optim\lr_scheduler.py:432: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
  0%|          | 1/500 [00:00<02:06,  3.94it/s]

epoch 0 | average train loss = 0.00243  and average validation loss = 0.01873  |learning rate = 0.01000


  5%|▌         | 27/500 [00:05<01:33,  5.04it/s]

epoch 25 | average train loss = 0.00065  and average validation loss = 0.00439  |learning rate = 0.00640


 10%|█         | 52/500 [00:10<01:29,  5.00it/s]

...

 90%|█████████ | 451/500 [01:30<00:09,  4.97it/s]

epoch 450 | average train loss = 0.00041  and average validation loss = 0.00348  |learning rate = 0.00134


 95%|█████████▌| 476/500 [01:35<00:04,  5.24it/s]

epoch 475 | average train loss = 0.00042  and average validation loss = 0.00342  |learning rate = 0.00134


100%|██████████| 500/500 [01:40<00:00,  5.00it/s]


In [16]:
frames = []

for t in range(num_epochs):
    image = imageio.v2.imread(f'./images/training/graph_{t}.png')
    frames.append(image)

imageio.mimsave('./example.gif', frames, fps=8)

In [1]:
df = pd.read_csv('./metrics/train_metrics.csv')
ax1 = df.plot.line(x='epoch', y='loss')

NameError: name 'pd' is not defined